In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from utils import viz_samples, Config
from noise_scheduler import NoiseScheduler

In [ ]:
cfg = {
    "seed": 0,

    # data
    "csv_file": "assets/simple/cat.csv",

    # opt_params
    "epochs": 5,
    "batch_size": 32,
    "lr": 1e-3,

    # model_params
    "model": "MLP", # denoiser
    "input_dim": 3,
    "hidden_dim": 64,
    "output_dim": 2,
    "hidden_layers": 3,

    # output_params
    "log_dir": "./logs/",

    # noise_scheduler
    "beta_start": 1e-5,
    "beta_end": 1e-2,
    "timesteps": 50,
}

In [ ]:
cfg = Config(cfg)
log_dir = cfg.log_dir
ns = NoiseScheduler(cfg)
model = torch.load(cfg.log_dir + "params.pt")
model.eval()

In [ ]:
'''
Run inference
'''
x_last = torch.randn(1, 2)
samples = [x_last]

for t in reversed(range(ns.num_timesteps)):
    with torch.no_grad():
        eps_pred = model(samples[-1], t)
        # reconstruct image here
        
    # add to samples
    samples.append()